In [23]:
import pandas as pd
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import datetime
import time
from scipy.stats import probplot
import datetime
import seaborn as sns
sns.set()
from sklearn.utils import shuffle
import random
from keras.preprocessing import sequence
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, TensorBoard, EarlyStopping
from keras.models import Sequential, Model
from keras.layers import (Dense, Dropout, BatchNormalization, LSTM,
                          GaussianNoise, Input, PReLU, Activation,
                          Concatenate, GRU, Masking, TimeDistributed,
                          Conv2D, MaxPooling2D, Flatten)
from keras.initializers import VarianceScaling
from keras import regularizers 
from keras.models import load_model
from keras import backend as K
from sklearn import metrics

In [2]:
def load_data():
    dframe_path = 'data/cabauw/processed-full-log.csv.gz'
    df = pd.read_csv(dframe_path, na_values='--', compression='gzip')

    df = df[(df.ustar > 0.1) & (abs(df.H) > 10) & (df.wind > 1)]
    df = df[df.ds != 201603]
    
    return df

df = load_data()

# test RNN

In [3]:
class RollingMeanStd:
    # http://people.ds.cam.ac.uk/fanf2/hermes/doc/antiforgery/stats.pdf
    def __init__(self):
        self.n = 0
        self.mu = self.s = None
    
    def add(self, x):
        self.n += 1
        if self.n == 1:
            self.mu = self.s = x
            return

        old_mu = self.mu
        old_s = self.s
        
        self.mu = old_mu + (x - old_mu) / self.n
        self.s = old_s + (x - old_mu) * (x - self.mu)
    
    @property
    def count(self):
        return self.n

    @property
    def rolling_mean(self):
        return self.mu
    
    @property
    def rolling_std(self):
        return np.sqrt(self.s / self.n)

In [4]:
wind_temp_levels = df.pivot_table(
    values=['wind', 'temp'], columns='z', index=['ds', 'tt']
).reset_index()
wind_temp_levels.columns = [
    '%s_%d' % (a, b) if b else a
    for a, b in wind_temp_levels.columns.values
]

ddf = df.merge(wind_temp_levels, on=['ds', 'tt'])

feature_sets = [
    [
        'z', 'wind', 'temp', 'soil_temp',
        'wind_10', 'wind_20', 'wind_40',
        'temp_10', 'temp_20', 'temp_40',
    ],
    ['soilheat'],
    ['netrad'],
    ['rain', 'dewpoint'],
]

all_features = [f for fset in feature_sets for f in fset]

In [5]:
from collections import deque

window_size = 1 * 60 * 60

queues = {z: deque() for z in ddf.z.unique()}
normalizer = RollingMeanStd()
samples_x, samples_y = [], []
train_mask = []
nonas = ddf.dropna().sort_values('datetime')
test_ds = np.random.choice(nonas.ds.unique(), 20, replace=False)
test_mask = nonas.ds.isin(test_ds)
max_len = 0

for i, row in enumerate(nonas.itertuples()):
    qq = queues[row.z]
    qq.append(row)
    while qq and row.datetime - qq[0].datetime > window_size:
        qq.popleft()

    sample = []
    for obs in qq:
        ss = [getattr(obs, ff) for ff in all_features]
        ss.append(row.datetime - obs.datetime)
        sample.append(ss)
    
    max_len = max(max_len, len(sample))
    if not test_mask.iloc[i]:
        normalizer.add(np.array(ss[:-1]))

    samples_x.append(np.array(sample))
    samples_y.append(row.phi_m)
    
    if (i + 1) % int(len(nonas) / 25) == 0:
        print(i + 1, int(100 * (i + 1) / len(nonas)))

51989 3
103978 7
155967 11
207956 15
259945 19
311934 23
363923 27
415912 31
467901 35
519890 39
571879 43
623868 47
675857 51
727846 55
779835 59
831824 63
883813 67
935802 71
987791 75
1039780 79
1091769 83
1143758 87
1195747 91
1247736 95
1299725 99


In [68]:
mu_x = normalizer.rolling_mean
std_x = normalizer.rolling_std

mu_y = np.mean(np.extract(test_mask, samples_y))
std_y = np.std(np.extract(test_mask, samples_y))

def make_batches(all_samples, epochs, batch_size, pad_to,
                 num_features, shuffle=True, adaptive_width=True):
    ep = 0

    batch_x = np.zeros((batch_size, pad_to, num_features))
    batch_y = np.zeros((batch_size, 1))

    while epochs < 0 or ep < epochs:
        if shuffle:
            random.shuffle(all_samples)

        ep += 1
        max_len = 0
        batch_x[:, :, -1] = -1

        for i, (sx, sy) in enumerate(all_samples):
            max_len = max(max_len, sx.shape[0])

            batch_x[i % batch_size, -sx.shape[0]:, :-1] = (sx[:, :-1] - mu_x) / std_x
            batch_x[i % batch_size, -sx.shape[0]:, -1] = sx[:, -1] / window_size
            batch_y[i % batch_size] = (sy - mu_y) / std_y

            assert np.all(np.isfinite(sx)), i
            assert np.all(np.isfinite(sy)), i

            if (i + 1) % batch_size == 0:
                if adaptive_width:
                    yield batch_x[:, -max_len:], batch_y
                else:
                    yield batch_x, batch_y

                max_len = 0
                batch_x[:, :, -1] = -1

        remaining = (i + 1) % batch_size
        if remaining > 0:
            if adaptive_width:
                yield batch_x[:remaining, -max_len:], batch_y[:remaining]
            else:
                yield batch_x[:remaining, :], batch_y[:remaining, :]


bx, by = next(make_batches(list(zip(samples_x, samples_y)), -1,
                           32, max_len, 15, adaptive_width=True))
bx.shape, by.shape

((32, 7, 15), (32, 1))

In [90]:
def compute_denormalized_mse(std_y):
    def denorm_mse(y_true, y_pred):
        # model is trained with normalized data, but we want
        # mse on not normalized data to compare with MOST
        mse = K.mean(K.square(y_true - y_pred), axis=-1)
        return mse * std_y**2
    return denorm_mse


def build_model():
    layer_kwargs = {
        'kernel_regularizer': regularizers.l2(1e-6),
        'activity_regularizer': regularizers.l2(1e-6),
        'activation': None
    }

    act = lambda: PReLU()

    model = Sequential([
        #GaussianNoise(0.05, input_shape=(None, 15)),
        GRU(128, input_shape=(None, 15), **layer_kwargs),
        act(),
        Dense(128, **layer_kwargs), act(),
        Dense(128, **layer_kwargs), act(),
        Dense(64, **layer_kwargs), act(),
        Dense(16, **layer_kwargs), act(),
        Dense(8, **layer_kwargs), act(),
        Dense(4, **layer_kwargs), act(),
        Dense(1)
    ])
    opt = Adam(clipnorm=10)
    model.compile(optimizer=opt, loss='mse',
                  metrics=[compute_denormalized_mse(std_y)])
    return model


build_model().count_params()
#build_model().summary()

98269

In [ ]:
batch_size = 1024
run_on_gpu = True

train_samples = list(zip(np.extract(~test_mask, samples_x),
                         np.extract(~test_mask, samples_y)))
#train_samples = random.sample(train_samples, 100000)
train_steps = int(len(train_samples) / batch_size) + 1
print('%d training samples, %d batches per epoch' % (len(train_samples), train_steps))


val_samples = list(zip(np.extract(test_mask, samples_x),
                       np.extract(test_mask, samples_y)))
#val_samples = random.sample(val_samples, 25000)
val_steps = int(len(val_samples) / batch_size) + 1
print('%d val. samples, %d batches per val.' % (len(val_samples), val_steps))


exec_id = (datetime.datetime
    .utcnow()
    .isoformat()
    .replace('-', '')
    .replace(':', '')
    .replace('T', '-')
)[:-7]

with tf.Graph().as_default():
    with tf.Session(config=tf.ConfigProto(
        device_count={'GPU': int(run_on_gpu)}
    )).as_default():
        
        model = build_model()
    
        train_data = make_batches(train_samples, -1, batch_size, max_len, 15)
        val_data = make_batches(val_samples, -1, batch_size, max_len, 15)
        model.fit_generator(
            generator=train_data,
            steps_per_epoch=train_steps,
            epochs=500,
            validation_data=val_data,
            validation_steps=val_steps,
            verbose=1,
            callbacks=[
                ReduceLROnPlateau(epsilon=0.001, verbose=1, min_lr=1e-6),
                TensorBoard('dev/logs/rnn-%s' % exec_id)
            ]
        )

# plain dense fc networks

In [18]:
def make_index(dtimes, interval):
    # returns a tuple index_above, index_below
    # index_above[i] is the largest i
    # such that dtimes[index_above[i]] - dtimes[i] < interval
    # index_below[i] is the smallest i
    # such that dtimes[i] - dtimes[index_below[i]] < interval
    # dtimes must be already sorted!
    index_below, index_above = np.zeros(
        (2, len(dtimes)), dtype=np.int
    ) - 1
    
    for i, x in enumerate(dtimes):
        j = index_below[i - 1] if i > 0 else 0
        while x - dtimes[j] > interval:
            j += 1

        index_below[i] = j
        index_above[j] = i

    last_above = index_above[0]
    for i in range(len(dtimes)):
        if index_above[i] < 0:
            index_above[i] = last_above
        else:
            last_above = index_above[i]
    
    return index_above, index_below


def compute_trend(df, columns, interval=3600):
    df = df.sort_values('datetime')
    for z in df.z.unique():  
        this_level = df[df.z == z]
        index_above, index_below = make_index(this_level.datetime.values, interval)

        for col in columns:
            val_above = this_level[col].values
            val_below = this_level.iloc[index_below][col].values

            time_above = this_level.datetime.values
            time_below = this_level.iloc[index_below].datetime.values

            trend = 3600 * (val_above - val_below) / (time_above - time_below)

            df.loc[df.z == z, col + '_trend'] = trend

    return df, [col + '_trend' for col in columns]


def get_features(df, use_trend, feature_level):
    wind_temp_levels = df.pivot_table(
        values=['wind', 'temp'], columns='z', index=['ds', 'tt']
    ).reset_index()
    wind_temp_levels.columns = [
        '%s_%d' % (a, b) if b else a
        for a, b in wind_temp_levels.columns.values
    ]

    df = df.merge(wind_temp_levels, on=['ds', 'tt'])

    feature_sets = [
        [
            'z', 'wind', 'temp', 'soil_temp',
            'wind_10', 'wind_20', 'wind_40',
            'temp_10', 'temp_20', 'temp_40',
        ],
        ['soilheat'],
        ['netrad'],
        ['rain', 'dewpoint'],
        ['H', 'LE'],
    ]

    features = [
        f for fset in feature_sets[:feature_level]
        for f in fset
    ]

    if use_trend:
        df, added_cols = compute_trend(df, [
            f for f in features if f != 'z'
        ])
        features.extend(added_cols)
        
    return df, features


def get_train_test_data_my_random_months(df, features, target, n_months=18):
    test_ds = np.random.choice(df.ds.unique(), n_months, replace=False)
    test_mask = df.ds.isin(test_ds)

    train_x, train_y = df.loc[~test_mask, features], df.loc[~test_mask, target]
    test_x, test_y = df.loc[test_mask, features], df.loc[test_mask, target]

    mean_x, mean_y = train_x.mean(), train_y.mean()
    std_x, std_y = train_x.std(), train_y.std()

    train_x = (train_x - mean_x) / std_x
    test_x = (test_x - mean_x) / std_x
    
    assert np.all(np.isfinite(train_x))
    assert np.all(np.isfinite(test_x))
    
    train_y = (train_y - mean_y) / std_y
    test_y = (test_y - mean_y) / std_y

    return features, train_x, train_y, test_x, test_y, mean_y, std_y

In [19]:
class AttributeKFold:
    ''' k-fold cross validator splitting on a particular attribute
        so that all samples with a given value are either in the train or test set

        attribute value for each sample is given in the constructor, so that
        the attribute itself need not be in the features for the model
    '''
    def __init__(self, cv, attr):
        self.cv, self.attr = cv, attr

    def get_n_splits(self, *args, **kwargs):
        return self.cv.get_n_splits(*args, **kwargs)

    def split(self, X, y=None, groups=None):
        vals = self.attr.unique()
        for train_idx, test_idx in self.cv.split(vals):
            train_mask = self.attr.isin(vals[train_idx])
            test_mask = self.attr.isin(vals[test_idx])

            X = np.argwhere(train_mask).reshape(-1)
            y = np.argwhere(test_mask).reshape(-1)
            
            assert np.all(np.isfinite(X))
            assert np.all(np.isfinite(y))
            
            yield X, y

In [20]:
def compute_denormalized_mse(std_y):
    def denormalized_mse(y_true, y_pred):
        # model is trained with normalized data, but we want
        # mse on not normalized data to compare with MOST
        mse = K.mean(K.square(y_true - y_pred), axis=-1)
        return mse * std_y**2
    return denormalized_mse


def orthonormal_regularizer(regu):
    # eq. 3 in https://arxiv.org/pdf/1703.01827.pdf
    def compute(weight_matrix):
        rows, cols = weight_matrix.shape
        wtw = K.dot(K.transpose(weight_matrix), weight_matrix)
        return regu * K.sum((wtw - K.eye(cols.value))**2) / 2
    return compute


def build_model(sizes, std_y=1):
    # every element in sizes specifies a layer
    #   negative number: skip connection of -n layers
    #                    successive skips are aggregated
    #                    into a single layer
    #   0<n<1: dropout with pkeep=n
    #   >1 fully connected then prelu
    layers = [Input(shape=(sizes[0],)),]
    #layers.append(GaussianNoise(0.01)(layers[-1]))
    i = 1
    while i < len(sizes):
        num = sizes[i]
        if num < 0:
            skip = [layers[-1]]
            while i < len(sizes) and sizes[i] < 0:
                skip.append(layers[sizes[i] - 1])
                i += 1
            layer = Concatenate()(skip)
            i -= 1
        elif num < 1:
            layer = Dropout(num)(layers[-1])
        else:
            layer = PReLU()(
                Dense(num, kernel_initializer=VarianceScaling(2, 'fan_in'))(
                    layers[-1]
                )
            )

        layers.append(layer)
        i += 1

    layers.append(Dense(1)(layers[-1]))

    opt = Adam(lr=0.001)
    model = Model(inputs=layers[0], outputs=layers[-1])
    model.compile(loss='mse', optimizer=opt, metrics=[compute_denormalized_mse(std_y)])
    return model

In [21]:
def run_experiment(network, use_trend, feature_level, batch_size=1024, verbose=2, most_only=False):
    ddf, features = get_features(df.dropna(), use_trend, feature_level)
    if most_only:
        ddf = ddf[(ddf.zL > -2) & (ddf.zL < 1)]
    features, train_x, train_y, test_x, test_y, mean_y, std_y = get_train_test_data_my_random_months(
        ddf, features, 'phi_m'
    )

    K.clear_session()  # https://stackoverflow.com/q/35114376/521776
    model = build_model([len(features)] + network, std_y=std_y)

    dtime = datetime.datetime.utcnow().isoformat().replace('-', '').replace(':', '').replace('T', '-')[:-7]
    logdir = 'dev/logs/%s-tren%s-features%s-batchsize%s-nparam%s/' % (
        dtime, use_trend, feature_level, batch_size, model.count_params()
    )

    if verbose > 0:
        print('Saving to', logdir)

    callbacks = [
        ReduceLROnPlateau(factor=0.1, verbose=verbose, min_lr=1e-6, patience=10),
        ModelCheckpoint(logdir + 'best.hdf5', verbose=verbose, save_best_only=True),
        TensorBoard(logdir, write_graph=True, write_grads=True, histogram_freq=0),
        EarlyStopping(min_delta=0.0001, patience=25),
    ]

    hist = model.fit(
        train_x, train_y,
        batch_size=batch_size,
        epochs=1000,
        verbose=verbose,
        shuffle=True,
        callbacks=callbacks,
        validation_data=(test_x, test_y)
    )

    return hist, logdir

In [22]:
def get_train_test_data_by_index(df, features, target, train_idx, test_idx, normalize):
    train_x, train_y = df.iloc[train_idx][features], df.iloc[train_idx][target]
    test_x, test_y = df.iloc[test_idx][features], df.iloc[test_idx][target]

    if normalize:
        mean_x, std_x = train_x.mean(), train_x.std()
        train_x = (train_x - mean_x) / std_x
        test_x = (test_x - mean_x) / std_x

        mean_y, std_y = train_y.mean(), train_y.std()
        train_y = (train_y - mean_y) / std_y
        test_y = (test_y - mean_y) / std_y
    else:
        mean_y, std_y = 0, 1

    return train_x, train_y, test_x, test_y, mean_y, std_y


def run_cv(network, use_trend, feature_level, most_only, batch_size=1024, cv_folds=5, verbose=0):
    ddf, features = get_features(df.dropna(), use_trend, feature_level)
    finite = np.isfinite(ddf[features]).all(axis=1)
    ddf = ddf.loc[finite]
    if most_only:
        ddf = ddf[(ddf.zL > -2) & (ddf.zL < 1)]

    cv = AttributeKFold(KFold(cv_folds, shuffle=True), ddf.ds)
    results = []
    for cv_idx, (train_idx, test_idx) in enumerate(cv.split(ddf.ds)):

        # prepare data
        train_x, train_y, test_x, test_y, mean_y, std_y = get_train_test_data_by_index(
            ddf, features, 'phi_m', train_idx, test_idx, normalize=True
        )

        K.clear_session()  # https://stackoverflow.com/q/35114376/521776
        model = build_model([len(features)] + network, std_y=std_y)
        dtime = datetime.datetime.utcnow().isoformat().replace('-', '').replace(':', '').replace('T', '-')[:-7]
        logdir = 'dev/logs/%s-tren%s-features%s-batchsize%s-nparam%s-cv%d/' % (
            dtime, use_trend, feature_level, batch_size, model.count_params(), cv_idx
        )
        save_to = logdir + 'best.hdf5'
        if verbose > 0:
            print('Saving to', logdir)

        # fit to train data
        hist = model.fit(
            train_x, train_y,
            batch_size=batch_size,
            epochs=500,
            verbose=verbose,
            shuffle=True,
            callbacks=[
                ReduceLROnPlateau(factor=0.1, verbose=verbose, min_lr=1e-6, patience=10),
                ModelCheckpoint(save_to, verbose=verbose, save_best_only=True),
                TensorBoard(logdir, write_graph=True, write_grads=True, histogram_freq=0),
                EarlyStopping(min_delta=0.0001, patience=25),
            ],
            validation_data=(test_x, test_y)
        )
        
        # evaluate on test data
        best = load_model(logdir + 'best.hdf5', custom_objects={
            'denormalized_mse': compute_denormalized_mse(std_y)
        })

        y_pred = best.predict(test_x)
        y_pred = y_pred * std_y + mean_y
        test_y = test_y * std_y + mean_y
        
        y_pred = y_pred.reshape(-1)
        test_y = test_y.values.reshape(-1)

        results.append((
            metrics.explained_variance_score(test_y, y_pred),
            metrics.mean_absolute_error(test_y, y_pred),
            metrics.mean_squared_error(test_y, y_pred),
            metrics.median_absolute_error(test_y, y_pred),
            metrics.r2_score(test_y, y_pred),
            np.mean(np.abs((test_y - y_pred) / test_y)) * 100,
        ))
    
    return pd.DataFrame(results, columns=[
        'explained_variance', 'mean_absolute_error', 'mean_squared_error',
        'median_absolute_error', 'r2_score', 'mean_absolute_percent_error'
    ])


def test_setting(use_trend, feature_level, most_only, batch_size=1024, cv_folds=10, verbose=0):
    # test all models on the given setting
    models = [
        [128, 64, 32, 16, 8, 4, 2, 1],
        [256, 0.5, 128, 64, 32, 16, 8, 4, 2, 1],
        [256, 0.5, 128, 64, 64, 32, 16, 8, 4, 2, 1], 
        [256, 0.5, 128, 64, 64, 32, 32, 16, 8, 4, 2, 1],
        [512, 0.5, 256, 0.5, 128, 64, 32, 16, 8, 4, 2, 1],
        [256, 0.5, 128, 64, 64, 32, 32, 16, 8, 4, 2, 1],
    ]

    results = []
    for i, mod in enumerate(models):
        res = run_cv(
            mod, use_trend, feature_level, most_only, batch_size, cv_folds, verbose
        )
        print('----  model', i)
        print(res.describe().T)
        results.append(res)
    return results

In [23]:
rt5m = test_setting(
    use_trend=True,
    feature_level=5,
    most_only=True,
)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide


----  model 0
                             count       mean        std        min  \
explained_variance            10.0   0.845168   0.007878   0.836138   
mean_absolute_error           10.0   0.223719   0.009633   0.209812   
mean_squared_error            10.0   0.128021   0.010448   0.114250   
median_absolute_error         10.0   0.139679   0.006043   0.131263   
r2_score                      10.0   0.845037   0.007892   0.836013   
mean_absolute_percent_error   10.0  66.382704  33.606492  43.173563   

                                   25%        50%        75%         max  
explained_variance            0.838149   0.843280   0.852672    0.855499  
mean_absolute_error           0.216687   0.223827   0.230371    0.237424  
mean_squared_error            0.119943   0.126748   0.133647    0.144310  
median_absolute_error         0.134649   0.138898   0.145076    0.147642  
r2_score                      0.837851   0.843267   0.852616    0.855380  
mean_absolute_percent_error  46.180267

TypeError: Could not operate array([1.21437722e+01, 2.68556354e+00, 6.13142094e+00, 5.47967716e+00,
       2.34921918e+00, 2.54741347e+00, 2.75082854e+00, 6.16887242e+00,
       6.12081704e+00, 6.07938917e+00, 1.27679213e+01, 1.60938388e+02,
       1.30745856e-01, 5.60341478e+00, 4.70466667e+01, 1.10642087e+02,
       1.33562232e+00, 7.89776570e-01, 2.82331789e-01, 1.24140715e+00,
       1.33764925e+00, 1.43687907e+00, 8.07513164e-01, 7.85138730e-01,
       7.63359860e-01, 4.75344300e+00, 8.41352168e+01, 2.21128149e-01,
       9.35291275e-01, 2.83899059e+01, 6.27276580e+01]) with block values 